In [2]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/p

In [3]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-23 06:08:45--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.96MB/s    in 0.2s    

2023-02-23 06:08:45 (4.96 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url= "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
kitchen_review_data_df= spark.read.csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
kitchen_review_data_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [6]:
# Get the number of rows in the DataFrame.
rows = kitchen_review_data_df.count()
print(f"DataFrame Rows count : {rows}")


DataFrame Rows count : 4880466


# Transform the Data

## Create the "review_id_table".

In [11]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = kitchen_review_data_df.select(["review_id","customer_id","product_id","product_parent",
                                          to_date("review_date","yyyy-MM-dd").alias("review_date")])
review_id_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3DT59XH7HXR9K|   37000337|B00303FI0G|     529320574| 2015-08-31|
|R1LFS11BNASSU8|   15272914|B00JCZKZN6|     274237558| 2015-08-31|
|R296RT05AG0AF6|   36137863|B00JLIKA5C|     544675303| 2015-08-31|
|R3V37XDZ7ZCI3L|   43311049|B000GBNB8G|     491599489| 2015-08-31|
|R14GU232NQFYX2|   13763148|B00VJ5KX9S|     353790155| 2015-08-31|
| RZQH4V7L2O1PL|   19009420|B00HYB5YY0|     432241873| 2015-08-31|
|R1F8JMOSPJ3KO7|   40599388|B000HEBAV2|     584680984| 2015-08-31|
|R1ZISGY2BWW4Z5|   22719359|B0012DS4GG|     772637306| 2015-08-31|
|R17PW4I3AE5WZW|   47478640|B00FLQ4EE6|     755416578| 2015-08-31|
|R3D93G1KTP6A8P|   34195504|B00DBS9OTG|     648762742| 2015-08-31|
|R18TQIW1NKPUNU|   19100570|B00AN9UJ68|     495720940| 2015-08-31|
|R34KUNL21WU248|   10299811|B00L2P0KNO|      41330497| 2015-08

## Create the "products" Table

In [17]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = kitchen_review_data_df.select(["product_id","product_title"]).drop_duplicates()

products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004NWK4W0|Wildkin Pink Leop...|
|B00IE70YFC|Wilton 415-2178 3...|
|B00004W499|George Foreman GG...|
|B000Q945FG|BIA Cordon Bleu B...|
|B007WTOIPA|Clorox Sales Co B...|
|B006IC09P0|Utenlid Acrylic R...|
|B00CDSU2MI|2000 Diamond Tabl...|
|B000WJMTNA|OXO Good Grips St...|
|B00DYRI13O|Reed & Barton 148...|
|B005FPD67Y|     Ergo Chef Tongs|
|B00JLA6POU|TeChef - Art Pan ...|
|B00FN3P7X4|Hamilton Beach 1....|
|B000UVHUCU|KitchenAid Ultra ...|
|B00IZCWYR2|PackIt Freezable ...|
|B0105V7VS2|Top Rated Lemon S...|
|B002P667TU|Browne (22126P) 6...|
|B00KIYWDTO|Wolfgang Puck Pre...|
|B00004S9EJ|Cuisinart DLC-8SY...|
|B00D19SSE2|Ozeri Serafino Do...|
|B00JV8PTYY|Thermos 12 Ounce ...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [13]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
# customers_df = toy_review_data_df.select(["customer_id","customer_count"])
# customers_df.show()
from pyspark.sql.functions import sum,avg,max,count
customers_df = kitchen_review_data_df.groupby('customer_id').agg(count("*").alias("customer_count"))
customers_df.show()



+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             1|
|   48875707|             2|
|   48088660|             1|
|   20781887|             6|
|    1075003|             1|
|   28377689|             3|
|    8338749|             1|
|   39271457|             6|
|   38209321|             5|
|   51397605|             1|
|   45337932|             7|
|   12948675|             3|
|   38892468|             1|
|   43582450|             1|
|   13156755|             1|
|   13433330|             1|
|   17067926|             7|
|   13780617|             1|
|   50607818|             1|
|   33437759|             2|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [14]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = kitchen_review_data_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3DT59XH7HXR9K|          5|            0|          0|   N|
|R1LFS11BNASSU8|          5|            0|          1|   N|
|R296RT05AG0AF6|          5|            0|          0|   N|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|
|R14GU232NQFYX2|          5|            0|          0|   N|
| RZQH4V7L2O1PL|          1|            1|          1|   N|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|
|R17PW4I3AE5WZW|          5|            0|          0|   N|
|R3D93G1KTP6A8P|          3|            0|          0|   N|
|R18TQIW1NKPUNU|          5|            0|          0|   N|
|R34KUNL21WU248|          4|            0|          0|   N|
|R2YA1ZA53X12IN|          5|            0|          0|   N|
|R2ZD1IGC9UU55X|          5|            

# Load

In [15]:
mode = "append"
jdbc_url="jdbc:postgresql://endpoint"
config = {"user":"root", "password": <password>, "driver":"org.postgresql.Driver"}

In [16]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


In [18]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)


In [19]:
# Write customers_df to table in RDS
customers_df .write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [20]:
# Write vine_df to table in RDS
vine_df .write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)